In [ ]:
# default_exp pde

# PDE

> This module contains the PDE class with the basic functionality to solve PDEs with NNs.

In [ ]:
#export

class PDE:
    "PDE class with basic functionality to solve PDEs with NNs"
    def __init__(self, inputs=None, outputs=None, params=[]):
        
        self.inputs = inputs
        self.outputs = outputs
        self.params = params
        
        # initialize values
        self.inputTrainValues = initStringList(self.inputs)
        self.inputValValues = initStringList(self.inputs)
        self.inputTestValues = initStringList(self.inputs)
        self.outputValues = initStringList(self.outputs)
        self.paramValues = initStringList(self.params)
        
        self.bocos = []

        
    def summary(self):
        "Print a summary of the PDE inputs, outputs, params and bocos."
        print('inputs (train): ', {name: values for name, values in zip(self.inputs, self.inputTrainValues)})
        print('inputs (val): ', {name: values for name, values in zip(self.inputs, self.inputValValues)})
        print('outputs: ', {name: values for name, values in zip(self.outputs, self.outputValues)})
        print('params: ', {name: values for name, values in zip(self.params, self.paramValues)})
        print('bocos: ', [boco.type for boco in self.bocos])
        print('')

In [ ]:
#hide
from nbdev.showdoc import *
show_doc(PDE.__init__)
show_doc(PDE.summary)

<h4 id="PDE.__init__" class="doc_header"><code>PDE.__init__</code><a href="__main__.py#L5" class="source_link" style="float:right">[source]</a></h4>

> <code>PDE.__init__</code>(**`inputs`**=*`None`*, **`outputs`**=*`None`*, **`params`**=*`[]`*)

Initialize self.  See help(type(self)) for accurate signature.

<h4 id="PDE.summary" class="doc_header"><code>PDE.summary</code><a href="__main__.py#L21" class="source_link" style="float:right">[source]</a></h4>

> <code>PDE.summary</code>()

Print a summary of the PDE inputs, outputs, params and bocos.

In [ ]:
#hide
from nangs.utils import *

You can instantiate a PDE as follows:

In [ ]:
pde = PDE(inputs=['x', 't'], outputs=['p'], params=['u'])

You can get a summary of the inputs, outputs, parameters and boundary conditions with the *summary* method.

In [ ]:
pde.summary()

inputs (train):  {'x': [], 't': []}
inputs (val):  {'x': [], 't': []}
outputs:  {'p': []}
params:  {'u': []}
bocos:  []

